In [1]:
import sys
from pathlib import Path
import site
import numpy as np
import pandas as pd
import torch
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

repo_root = Path('/home/dhanush/work/3S-Testing').resolve()
repo_synthcity = repo_root / 'synthcity'
sys.path = [p for p in sys.path if str(repo_synthcity) not in str(p)]
for sp in reversed(site.getsitepackages() if hasattr(site, 'getsitepackages') else []):
    if sp in sys.path:
        sys.path.remove(sp)
    sys.path.insert(0, sp)
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

from synthcity.plugins import Plugins
import synthcity
from src.data_loader import load_adult_data

seed = 0
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)

print('synthcity from', synthcity.__file__)
print('DDPM available:', 'ddpm' in Plugins().list())

[KeOps] Warning : CUDA libraries not found or could not be loaded; Switching to CPU only.


[2025-12-05T22:57:39.359882-0500][58275][CRITICAL] module disabled: /home/dhanush/anaconda3/envs/3s/lib/python3.9/site-packages/synthcity/plugins/generic/plugin_goggle.py
[2025-12-05T22:57:39.360869-0500][58275][CRITICAL] load failed: 
arfpy is not installed. Please install it with pip install arfpy.
Please be aware that arfpy is only available for python >= 3.8.

[2025-12-05T22:57:39.361217-0500][58275][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_arf' has no attribute 'plugin'
[2025-12-05T22:57:39.361690-0500][58275][CRITICAL] module plugin_arf load failed
[2025-12-05T22:57:39.360869-0500][58275][CRITICAL] load failed: 
arfpy is not installed. Please install it with pip install arfpy.
Please be aware that arfpy is only available for python >= 3.8.

[2025-12-05T22:57:39.361217-0500][58275][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_arf' has no attribute 'plugin'
[2025-12-05T22:57:39.361690-0500][58275][CRITICAL] module plugin_arf load failed
[20

synthcity from /home/dhanush/anaconda3/envs/3s/lib/python3.9/site-packages/synthcity/__init__.py
DDPM available: True
DDPM available: True


# Load data

In [2]:
dataset_name = 'adult'
X_train, X_oracle, y_train, y_test, X, y = load_adult_data()
Data = X.copy()
Data['y'] = y
# matching original notebook split ratios
X_train, X_oracle = train_test_split(Data, test_size=0.65, random_state=seed)
X_train, X_test = train_test_split(X_train, test_size=0.2, random_state=seed)
X_train, X_hp = train_test_split(X_train, test_size=0.1, random_state=seed)
print('Data shapes:', Data.shape, X_train.shape, X_test.shape)

Data shapes: (45222, 12) (11394, 12) (3166, 12)


# Train Generators

In [3]:
discrete_columns = [
    'education-num','marital-status','employment_type','relationship','race','sex','country','y'
]
use_trained_model = False
n_generators = 3
ddpms = []

class DDPMSampler:
    def __init__(self, model):
        self.model = model
    def sample(self, n, condition_column=None, condition_value=None):
        gen_count = max(n, int(n*2)) if condition_column is not None else n
        try:
            out = self.model.generate(count=gen_count)
        except Exception:
            try:
                out = self.model.sample(count=gen_count)
            except Exception as e:
                raise RuntimeError('Generation failed: ' + str(e))
        try:
            df = out.dataframe()
        except Exception:
            df = pd.DataFrame(out)
        if condition_column is not None and condition_value is not None:
            df = df[df[condition_column] == condition_value]
        return df.head(n)

if not use_trained_model:
    base = Plugins().get('ddpm')
    for i in range(n_generators):
        model = base() if callable(base) else type(base)()
        try:
            model.fit(X_train, discrete_columns=discrete_columns, n_iterations=50, seed=seed+i)
        except TypeError:
            try:
                model.fit(X_train, discrete_columns=discrete_columns, iterations=50, seed=seed+i)
            except TypeError:
                model.fit(X_train, discrete_columns=discrete_columns, seed=seed+i)
        ddpms.append(model)
        print(f'Trained DDPM {i+1}/{n_generators}')
    ddpm_samplers = [DDPMSampler(m) for m in ddpms]
    ddpm_sampler = ddpm_samplers[0]
else:
    loaded = Plugins().get('ddpm')
    ddpm_sampler = DDPMSampler(loaded.load(f'../models/ddpm_{dataset_name}'))
print('Sampler ready')

[2025-12-05T22:57:40.131737-0500][58275][CRITICAL] module disabled: /home/dhanush/anaconda3/envs/3s/lib/python3.9/site-packages/synthcity/plugins/generic/plugin_goggle.py
[2025-12-05T22:57:40.132363-0500][58275][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_arf' has no attribute 'plugin'
[2025-12-05T22:57:40.132756-0500][58275][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_arf' has no attribute 'plugin'
[2025-12-05T22:57:40.133058-0500][58275][CRITICAL] module plugin_arf load failed
[2025-12-05T22:57:40.133457-0500][58275][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
[2025-12-05T22:57:40.133764-0500][58275][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_great' has no attribute 'plugin'
[2025-12-05T22:57:40.134057-0500][58275][CRITICAL] module plugin_great load failed
[2025-12-05T22:57:40.134472-0500][58275][CRITICAL] load failed: module 'synthcity.plugins.generic.plugin_bayesian_net

Trained DDPM 1/3


Epoch: 100%|██████████| 1000/1000 [03:42<00:00,  4.50it/s, loss=0.883]



Trained DDPM 2/3


Epoch: 100%|██████████| 1000/1000 [03:25<00:00,  4.86it/s, loss=0.883]

Trained DDPM 3/3
Sampler ready


# Train the downstream predictors

In [4]:
model_dict = {
    'mlp': MLPClassifier(random_state=seed),
    'knn': KNeighborsClassifier(),
    'dt': DecisionTreeClassifier(random_state=seed),
    'rf': RandomForestClassifier(random_state=seed),
    'gbc': GradientBoostingClassifier(random_state=seed),
    'bag': BaggingClassifier(random_state=seed),
    'ada': AdaBoostClassifier(random_state=seed),
    'svm': SVC(random_state=seed),
    'lr': LogisticRegression(random_state=seed),
}

def train_models(df, model_dict):
    trained = {}
    for name, clf in model_dict.items():
        trained[name] = clf.fit(df.drop('y', axis=1), df['y'])
    return trained

trained_model_dict = train_models(X_train, model_dict)
print('Trained downstream predictors')

Trained downstream predictors


# Run analysis

In [5]:
column_metric = 'race'
groups = list(np.unique(Data[column_metric]))


def evaluate_models(data):
    results = {}
    for g in groups:
        results[g] = {'acc': {}}
        subset = data[data[column_metric] == g]
        if len(subset) == 0:
            for m in trained_model_dict.keys():
                results[g]['acc'][m] = 0.0
            continue
        for m, clf in trained_model_dict.items():
            try:
                y_pred = clf.predict(subset.drop('y', axis=1))
                results[g]['acc'][m] = accuracy_score(subset['y'], y_pred)
            except Exception:
                results[g]['acc'][m] = 0.0
    return results


def run_analysis(random_state=0):
    X_tr, X_or = train_test_split(Data, test_size=0.65, random_state=0)
    X_tr, X_te = train_test_split(X_tr, test_size=0.2, random_state=random_state)

    oracle_results = evaluate_models(X_or)
    oracle_samples = {g: X_or[X_or[column_metric]==g].shape[0] for g in groups}

    test_results = evaluate_models(X_te)
    test_samples = {g: X_te[X_te[column_metric]==g].shape[0] for g in groups}

    synth_df_list = []
    for g in groups:
        synth_df = ddpm_sampler.sample(len(X_tr), condition_column=column_metric, condition_value=g)
        synth_df_list.append(synth_df)
    synth_data = pd.concat(synth_df_list, ignore_index=True) if len(synth_df_list)>0 else pd.DataFrame()

    if len(synth_data) > 0:
        synth_results = evaluate_models(synth_data)
        synth_samples = {g: synth_data[synth_data[column_metric]==g].shape[0] for g in groups}
    else:
        synth_results = {g:{'acc':{m:0.0 for m in trained_model_dict.keys()}} for g in groups}
        synth_samples = {g:0 for g in groups}

    aug_data = pd.concat([X_te, synth_data], ignore_index=True) if len(synth_data)>0 else X_te
    aug_results = evaluate_models(aug_data)
    aug_samples = {g: aug_data[aug_data[column_metric]==g].shape[0] for g in groups}

    return test_results, test_samples, oracle_results, oracle_samples, synth_results, synth_samples, aug_results, aug_samples

n_runs = 3
test_acc_list=[]; oracle_acc_list=[]; synth_acc_list=[]; aug_acc_list=[]
test_samples_list=[]; oracle_samples_list=[]; synth_samples_list=[]; aug_samples_list=[]
for i in tqdm(range(n_runs)):
    res = run_analysis(random_state=i*100)
    (tres, tsamp, ores, osamp, sres, ssamp, ares, asamp) = res
    test_acc_list.append(tres); oracle_acc_list.append(ores); synth_acc_list.append(sres); aug_acc_list.append(ares)
    test_samples_list.append(tsamp); oracle_samples_list.append(osamp); synth_samples_list.append(ssamp); aug_samples_list.append(asamp)
print('Completed', n_runs, 'runs')

100%|██████████| 3/3 [28:23<00:00, 567.91s/it]

Completed 3 runs


# Process results and save

In [7]:
metrics = ['acc']
models = ['rf']

for metric in metrics:
    for model in models:
        props = [count/np.sum(np.unique(Data[column_metric], return_counts=True)[1]) for count in np.unique(Data[column_metric], return_counts=True)[1]]
        df = pd.DataFrame(columns=['Group','Oracle','Synthetic','Augmented','Test'])
        df_std = pd.DataFrame(columns=['Group','Oracle','Synthetic','Augmented','Test'])
        for idx, g in enumerate(np.argsort(props)[::-1]):
            oracle_res = np.array([o[g][metric][model] for o in oracle_acc_list])
            synth_res = np.array([s[g][metric][model] for s in synth_acc_list])
            aug_res = np.array([a[g][metric][model] for a in aug_acc_list])
            test_res = np.array([t[g][metric][model] for t in test_acc_list])
            df = pd.concat([df, pd.DataFrame([{
                'Group': f"{idx+1} ({int(round(props[g]*100,0))}%)",
                'Oracle': round(np.mean(oracle_res)*100,2),
                'Synthetic': round(np.mean(synth_res)*100,2),
                'Augmented': round(np.mean(aug_res)*100,2),
                'Test': round(np.mean(test_res)*100,2),
            }])], ignore_index=True)
            df_std = pd.concat([df_std, pd.DataFrame([{
                'Group': f"{idx+1} ({int(round(props[g]*100,0))}%)",
                'Oracle': round(np.std(oracle_res)*100,2),
                'Synthetic': round(np.std(synth_res)*100,2),
                'Augmented': round(np.std(aug_res)*100,2),
                'Test': round(np.std(test_res)*100,2),
            }])], ignore_index=True)
        out1 = f"../results/{dataset_name}_subgroup_shift_ddpm_env_{model}_{metric}.csv"
        out2 = f"../results/{dataset_name}_subgroup_shift_ddpm_env_std_{model}_{metric}.csv"
        df.to_csv(out1, index=False)
        df_std.to_csv(out2, index=False)
        print('Saved', out1)
        print(df)

Saved ../results/adult_subgroup_shift_ddpm_env_rf_acc.csv
     Group  Oracle  Synthetic  Augmented   Test
0  1 (86%)   80.32      78.36      79.93  87.25
1   2 (9%)   88.72      88.82      89.44  95.36
2   3 (3%)   80.21      81.64      82.37  89.44
3   4 (1%)   87.63      88.75      89.08  93.10
4   5 (1%)   88.74      92.54      92.68  94.20
